In [ ]:
from utils.io import gen_id_dated, gdf_viewer, gdf_geom, gdf_merger, na_col_drop, na_line_drop, data_validation
import pandas as pd
import numpy as np
from utils.utils import update_dict

### Create data

In [ ]:
t1 = pd.DataFrame({'ID':['id1','id2','id3','id4','id5','id6','id7','id8', 'id9'],'Name':['A','B','C','D','E','F','G','H','I'],
                   'Value':[1,2,2,4,np.nan,5,np.nan,8,9], 
                   'Date': ['2021-06-01','2021-06-02','2021-06-04','2021-06-07','2021-06-08','2021-06-10',
                            '2021-06-11','2021-06-04','2021-06-02'],
                   'X': [1.,4.,50.,8.,7.,11.,8.,np.nan,10], 
                   'Y': [7.,4.,2.,16.,14.,20.,11.,np.nan,6]})

In [ ]:
t2 = pd.DataFrame({'ID':['id1','id2','id3','id4','id5','id6','id7','id8', 'id9'],'Name':['A','B','D','C',np.nan,'F','G','h','k'],
                   'Value':[1,3,2,2,np.nan,np.nan,6,7,9],
                   'Date': ['2021-06-01','2021-06-03','2021-06-05','2021-06-07','2021-06-09','2021-06-11',
                            '2021-06-03','2021-06-04',np.nan],
                   'X': [np.nan,4.,12.,8.,7.,11.,8.,13.,10], 
                   'Y': [np.nan,4.,20.,18.,104.,9.,1.,5.,6]})

In [ ]:
t1.merge(t2, on='ID', how='outer')

### Merging

In [ ]:
m, conflicts = gdf_merger(t1, t2, on='ID', how='outer', dist_max=1., date_col='Date')

In [ ]:
#m['index'] = m['index'].astype('int')
m

### conflicts management

In [ ]:
conflicts

In [ ]:
data_validation(overall_data=m, conflict_data=conflicts, valid_dict={'Value_x':[1,7], 'Value_y':[3],
                                                                    'Name_y':[2,3], 'Date_x':[1, 5]})

### Results

In [ ]:
m

In [ ]:
conflicts